In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date, lit

In [0]:
df_consumos = (
    spark.read.table("production.raw.raw_tb_consumos")
    .select(
        F.col("consumo_id"),
        F.col("reserva_id"),
        F.col("hospede_id"),
        F.col("hotel_id"),
        F.col("quarto_id"),
        F.col("data_consumo"),
        F.col("tipo_servico"),
        F.col("descricao"),
        F.round(F.col("valor").cast("double"), 2).alias("valor"),  
        F.col("pago")
    )
    .withColumn("data_consumo", F.to_date(F.col("data_consumo")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
df_faturas = (
    spark.read.table("production.raw.raw_tb_faturas")
    .select(
        F.col("fatura_id"),
        F.col("reserva_id"),
        F.col("hotel_id"),
        F.col("data_emissao"),
        F.col("data_vencimento"),
        F.col("data_pagamento"),
        F.round(F.col("valor_diarias").cast("double"), 2).alias("valor_diarias"),  
        F.round(F.col("subtotal_consumos").cast("double"), 2).alias("subtotal_consumos"),  
        F.round(F.col("impostos").cast("double"), 2).alias("impostos"),
        F.round(F.col("valor_total").cast("double"), 2).alias("valor_total"),
        F.col("forma_pagamento"),
        F.col("tipo_pagamento"),
        F.col("status_pagamento"),
        F.col("nota_fiscal")
    )
    .withColumn("data_emissao", F.to_date(F.col("data_emissao")))
    .withColumn("data_vencimento", F.to_date(F.col("data_vencimento")))
    .withColumn("data_pagamento", F.to_date(F.col("data_pagamento")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_faturas.limit(5))


In [0]:
# Importando as funções do PySpark (boa prática garantir que F esteja definido)
import pyspark.sql.functions as F

# 1. Aplicar os aliases (apelidos) aos DataFrames antes do join.
# Esta é uma forma limpa e legível de preparar a operação.
c = df_consumos.alias("c")
f = df_faturas.alias("f")

# 2. Executar o join e a seleção com a sintaxe correta
df_faturas_consumos = (
    c.join(f, on="reserva_id", how="inner") # 'how="inner"' é o padrão, mas é bom ser explícito.
    .select(
        # Colunas da tabela de consumos (alias 'c')
        F.col("c.consumo_id"),
        F.col("c.reserva_id"),
        F.col("c.hospede_id"),
        F.col("c.hotel_id"),
        F.col("c.quarto_id"),
        F.col("c.data_consumo"),
        F.col("c.tipo_servico"),
        F.col("c.descricao"),
        F.col("c.valor"),
        F.col("c.pago"),
        
        # Colunas da tabela de faturas (alias 'f')
        F.col("f.fatura_id"),
        F.col("f.data_emissao"),
        F.col("f.data_vencimento"),
        F.col("f.data_pagamento"),
        F.col("f.valor_diarias"),
        F.col("f.subtotal_consumos"),
        F.col("f.impostos"),
        F.col("f.valor_total"),
        F.col("f.forma_pagamento"),
        F.col("f.tipo_pagamento"),
        F.col("f.status_pagamento"),
        F.col("f.nota_fiscal")
    )
    # 3. Adicionar colunas de auditoria/controle no final
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

# Exibir o resultado para verificação
display(df_faturas_consumos.limit(5))

In [0]:
catalog_table = "production.trusted.tb_faturas_consumos"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_faturas_consumos.alias("source"),
        "target.consumo_id = source.consumo_id AND target.fatura_id = source.fatura_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_faturas_consumos.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_faturas_consumos
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )